In [1]:
import numpy as np
import os
from matplotlib.pyplot import imread
import datetime
import os
import warnings
warnings.filterwarnings("ignore")
import abc
from sys import getsizeof

In [2]:

np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

In [15]:
pip install keras

You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [1]:
pip install tensorflow

You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
train_doc = np.random.permutation(open('datasets/Project_data/train.csv').readlines())
val_doc = np.random.permutation(open('datasets/Project_data/val.csv').readlines())
batch_size = 10

Generator

In [4]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height

def generator(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,x)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    temp = imresize(image,(120,120))
                    temp = temp/127.5-1 #Normalize data
                    
                    batch_data[folder,idx,:,:,0] = (temp[:,:,0]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = (temp[:,:,1]) #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = (temp[:,:,2]) #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,x,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): 
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    temp = imresize(image,(120,120))
                    temp = temp/127.5-1 #Normalize data
                    
                    batch_data[folder,idx,:,:,0] = (temp[:,:,0])
                    batch_data[folder,idx,:,:,1] = (temp[:,:,1])
                    batch_data[folder,idx,:,:,2] = (temp[:,:,2])
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels

In [5]:
curr_dt_time = datetime.datetime.now()
train_path = 'datasets/Project_data/train'
val_path = 'datasets/Project_data/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 10 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 10


In [6]:
from scipy.misc import imresize

In [29]:
pip install scipy==1.1.0

You should consider upgrading via the '/usr/bin/python -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


#MODEL

In [7]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation, Dropout
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
import keras


#write your model here
#model a
model_a = Sequential()

model_a.add(Conv3D(8, #number of filters 
                 kernel_size=(3,3,3), 
                 input_shape=(30, 120, 120, 3),
                 padding='same'))
model_a.add(BatchNormalization())
model_a.add(Activation('relu'))

model_a.add(MaxPooling3D(pool_size=(2,2,2)))

model_a.add(Conv3D(16, #Number of filters, 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_a.add(BatchNormalization())
model_a.add(Activation('relu'))

model_a.add(MaxPooling3D(pool_size=(2,2,2)))

model_a.add(Conv3D(32, #Number of filters 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_a.add(BatchNormalization())
model_a.add(Activation('relu'))

model_a.add(MaxPooling3D(pool_size=(2,2,2)))

model_a.add(Conv3D(64, #Number pf filters 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_a.add(BatchNormalization())
model_a.add(Activation('relu'))

model_a.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_a.add(Flatten())

model_a.add(Dense(1000, activation='relu'))
model_a.add(Dropout(0.5))

model_a.add(Dense(500, activation='relu'))
model_a.add(Dropout(0.5))

#softmax layer
model_a.add(Dense(5, activation='softmax'))

In [8]:
optimiser = optimizers.Adam(lr=0.001) #write your optimizer
model_a.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_a.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d (Conv3D)              (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization (BatchNo (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation (Activation)      (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_1 (Batch (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_1 (Activation)    (None, 15, 60, 60, 16)    0

In [9]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [10]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, cooldown=1, verbose=1) # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

In [11]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

In [12]:
model_a.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Instructions for updating:
Please use Model.fit, which supports generators.
Source path =  datasets/Project_data/train ; batch size = 10
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 4.8181 - categorical_accuracy: 0.2670Source path =  datasets/Project_data/val ; batch size = 10

Epoch 00001: saving model to model_init_2021-07-0516_22_40.984452/model-00001-4.81811-0.26697-1.99632-0.25000.h5
67/67 [==============================] - 90s 1s/step - loss: 4.8181 - categorical_accuracy: 0.2670 - val_loss: 1.9963 - val_categorical_accuracy: 0.2500
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 2.3171 - categorical_accuracy: 0.3134
Epoch 00002: saving model to model_init_2021-07-0516_22_40.984452/model-00002-2.31711-0.31343-2.03804-0.30000.h5
67/67 [==============================] - 38s 567ms/step - loss: 2.3171 - categorical_accuracy: 0.3134 - val_loss: 2.0380 - val_categorical_accuracy: 0.3000
Epoch 3/10
67/67 [==============================] - ETA: 

In [13]:
classes = 5 #left swipe, right swipe, thumbs up, thumbs down, stop
channel = 3
x = 30 # number of frames
y = 120 # image width
z = 120 # image height

def generator_ex(source_path, folder_list, batch_size):
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [x for x in range(0,x)] #create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,x,y,z,channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,classes)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    temp = imresize(image,(y,z))
                    #Converting to gray scale
                    temp = temp.mean(axis=-1,keepdims=1) 
                    temp = temp/127.5-1 #Normalize data
                    batch_data[folder,idx] = temp #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
                
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        if (len(folder_list) != batch_size*num_batches):
            print("Batch: ",num_batches+1,"Index:", batch_size)
            batch_size = len(folder_list) - (batch_size*num_batches)
            batch_data = np.zeros((batch_size,x,y,z,channel)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,classes)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    temp = imresize(image,(y,z))
                    #Converting to gray scale
                    temp = temp.mean(axis=-1,keepdims=1) 
                    temp = temp/127.5-1 #Normalize data
                    
                    batch_data[folder,idx] = temp
                   
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels


In [13]:
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam

# Define model b
model_b = Sequential()
model_b.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x,y,z,channel), padding='same'))
model_b.add(Activation('relu'))
model_b.add(Conv3D(32, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_b.add(Dropout(0.25))

model_b.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_b.add(Dropout(0.25))

model_b.add(Flatten())
model_b.add(Dense(512, activation='relu'))
model_b.add(Dropout(0.5))
model_b.add(Dense(classes, activation='softmax'))

model_b.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_b.summary()

NameError: name 'channel' is not defined

In [14]:
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)

In [16]:
model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  datasets/Project_data/train ; batch size = 10
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 1.9993 - categorical_accuracy: 0.2021Source path =  datasets/Project_data/val ; batch size = 10

Epoch 00001: saving model to model_init_2021-07-0511_47_52.386877/model-00001-1.99928-0.20211-1.60951-0.18000.h5
67/67 [==============================] - 97s 1s/step - loss: 1.9993 - categorical_accuracy: 0.2021 - val_loss: 1.6095 - val_categorical_accuracy: 0.1800
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 1.6104 - categorical_accuracy: 0.1692
Epoch 00002: saving model to model_init_2021-07-0511_47_52.386877/model-00002-1.61044-0.16915-1.60959-0.17000.h5
67/67 [==============================] - 41s 605ms/step - loss: 1.6104 - categorical_accuracy: 0.1692 - val_loss: 1.6096 - val_categorical_accuracy: 0.1700
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 1.6099 - categorical_accuracy: 0.2488
Epoch 00003: saving model t

Changing x,y,z values Experiment 

In [17]:
x = 30 # number of frames
y = 60 # image width
z = 60 # image height

In [18]:
# Define model b
model_b = Sequential()
model_b.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x,y,z,channel), padding='same'))
model_b.add(Activation('relu'))
model_b.add(Conv3D(32, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_b.add(Dropout(0.25))

model_b.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
model_b.add(Activation('relu'))
model_b.add(MaxPooling3D(pool_size=(3, 3, 3), padding='same'))
model_b.add(Dropout(0.25))

model_b.add(Flatten())
model_b.add(Dense(512, activation='relu'))
model_b.add(Dropout(0.5))
model_b.add(Dense(classes, activation='softmax'))

model_b.compile(optimizer=keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_b.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_8 (Conv3D)            (None, 30, 60, 60, 32)    2624      
_________________________________________________________________
activation_8 (Activation)    (None, 30, 60, 60, 32)    0         
_________________________________________________________________
conv3d_9 (Conv3D)            (None, 30, 60, 60, 32)    27680     
_________________________________________________________________
activation_9 (Activation)    (None, 30, 60, 60, 32)    0         
_________________________________________________________________
max_pooling3d_6 (MaxPooling3 (None, 10, 20, 20, 32)    0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 10, 20, 20, 32)    0         
_________________________________________________________________
conv3d_10 (Conv3D)           (None, 10, 20, 20, 64)   

In [19]:
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)


In [20]:
model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  datasets/Project_data/train ; batch size = 10
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 1.6363 - categorical_accuracy: 0.2036Source path =  datasets/Project_data/val ; batch size = 10

Epoch 00001: saving model to model_init_2021-07-0511_47_52.386877/model-00001-1.63632-0.20362-1.55215-0.26000.h5
67/67 [==============================] - 80s 1s/step - loss: 1.6363 - categorical_accuracy: 0.2036 - val_loss: 1.5521 - val_categorical_accuracy: 0.2600
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 1.4989 - categorical_accuracy: 0.3284
Epoch 00002: saving model to model_init_2021-07-0511_47_52.386877/model-00002-1.49888-0.32836-1.43041-0.31000.h5
67/67 [==============================] - 36s 530ms/step - loss: 1.4989 - categorical_accuracy: 0.3284 - val_loss: 1.4304 - val_categorical_accuracy: 0.3100
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 1.4692 - categorical_accuracy: 0.2886
Epoch 00003: saving model t

Changing Batch size to 20 Experiment

In [21]:
train_generator = generator_ex(train_path, train_doc, 20)
val_generator = generator_ex(val_path, val_doc, 20)

In [22]:
model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  datasets/Project_data/train ; batch size = 20
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 0.5708 - categorical_accuracy: 0.7808Source path =  datasets/Project_data/val ; batch size = 20

Epoch 00001: saving model to model_init_2021-07-0511_47_52.386877/model-00001-0.57080-0.78084-0.57051-0.77000.h5
67/67 [==============================] - 105s 2s/step - loss: 0.5708 - categorical_accuracy: 0.7808 - val_loss: 0.5705 - val_categorical_accuracy: 0.7700
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 0.5409 - categorical_accuracy: 0.8209
Epoch 00002: saving model to model_init_2021-07-0511_47_52.386877/model-00002-0.54092-0.82090-0.53144-0.82000.h5
67/67 [==============================] - 49s 729ms/step - loss: 0.5409 - categorical_accuracy: 0.8209 - val_loss: 0.5314 - val_categorical_accuracy: 0.8200
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 0.6485 - categorical_accuracy: 0.7164
Epoch 00003: saving model 

Changing Batch size to 30 Experiment

In [23]:
train_generator = generator_ex(train_path, train_doc, 30)
val_generator = generator_ex(val_path, val_doc, 30)

In [24]:
model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  datasets/Project_data/train ; batch size = 30
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 0.1854 - categorical_accuracy: 0.9258Source path =  datasets/Project_data/val ; batch size = 30
Batch:  4 Index: 30

Epoch 00001: saving model to model_init_2021-07-0511_47_52.386877/model-00001-0.18541-0.92579-0.42341-0.85625.h5
67/67 [==============================] - 101s 2s/step - loss: 0.1854 - categorical_accuracy: 0.9258 - val_loss: 0.4234 - val_categorical_accuracy: 0.8562
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 0.3255 - categorical_accuracy: 0.8756
Epoch 00002: saving model to model_init_2021-07-0511_47_52.386877/model-00002-0.32550-0.87562-0.45815-0.83000.h5
67/67 [==============================] - 36s 535ms/step - loss: 0.3255 - categorical_accuracy: 0.8756 - val_loss: 0.4582 - val_categorical_accuracy: 0.8300
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 0.2604 - categorical_accuracy: 0.8706
Epoch 

Changing Batch size to 40 Experiment

In [25]:
train_generator = generator_ex(train_path, train_doc, 40)
val_generator = generator_ex(val_path, val_doc, 40)

In [26]:
model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  datasets/Project_data/train ; batch size = 40
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 0.0859 - categorical_accuracy: 0.9762Source path =  datasets/Project_data/val ; batch size = 40
Batch:  3 Index: 40

Epoch 00001: saving model to model_init_2021-07-0511_47_52.386877/model-00001-0.08594-0.97623-0.41888-0.84583.h5
67/67 [==============================] - 190s 3s/step - loss: 0.0859 - categorical_accuracy: 0.9762 - val_loss: 0.4189 - val_categorical_accuracy: 0.8458
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 0.0756 - categorical_accuracy: 0.9777
Epoch 00002: saving model to model_init_2021-07-0511_47_52.386877/model-00002-0.07558-0.97768-0.41364-0.87500.h5
67/67 [==============================] - 147s 2s/step - loss: 0.0756 - categorical_accuracy: 0.9777 - val_loss: 0.4136 - val_categorical_accuracy: 0.8750
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 0.0871 - categorical_accuracy: 0.9684
Epoch 00

Change Optimizer to Adadelta Experiment

In [27]:
model_b.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_b.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_8 (Conv3D)            (None, 30, 60, 60, 32)    2624      
_________________________________________________________________
activation_8 (Activation)    (None, 30, 60, 60, 32)    0         
_________________________________________________________________
conv3d_9 (Conv3D)            (None, 30, 60, 60, 32)    27680     
_________________________________________________________________
activation_9 (Activation)    (None, 30, 60, 60, 32)    0         
_________________________________________________________________
max_pooling3d_6 (MaxPooling3 (None, 10, 20, 20, 32)    0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 10, 20, 20, 32)    0         
_________________________________________________________________
conv3d_10 (Conv3D)           (None, 10, 20, 20, 64)   

In [28]:
train_generator = generator_ex(train_path, train_doc, 40)
val_generator = generator_ex(val_path, val_doc, 40)

In [29]:
model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  datasets/Project_data/train ; batch size = 40
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 0.0662 - categorical_accuracy: 0.9809Source path =  datasets/Project_data/val ; batch size = 40
Batch:  3 Index: 40

Epoch 00001: saving model to model_init_2021-07-0511_47_52.386877/model-00001-0.06622-0.98087-0.44037-0.86667.h5
67/67 [==============================] - 191s 3s/step - loss: 0.0662 - categorical_accuracy: 0.9809 - val_loss: 0.4404 - val_categorical_accuracy: 0.8667
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 0.0697 - categorical_accuracy: 0.9751
Epoch 00002: saving model to model_init_2021-07-0511_47_52.386877/model-00002-0.06972-0.97511-0.40241-0.86500.h5
67/67 [==============================] - 134s 2s/step - loss: 0.0697 - categorical_accuracy: 0.9751 - val_loss: 0.4024 - val_categorical_accuracy: 0.8650
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 0.0640 - categorical_accuracy: 0.9798
Epoch 00

Change epoch to 20 Experiment

In [30]:
num_epochs = 20 # choose the number of epochs
print ('# epochs =', num_epochs)

# epochs = 20


In [ ]:
import keras
model_b.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_b.summary()

In [32]:

train_generator = generator_ex(train_path, train_doc, 40)
val_generator = generator_ex(val_path, val_doc, 40)

In [33]:

model_b.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  datasets/Project_data/train ; batch size = 40
Epoch 1/20
67/67 [==============================] - ETA: 0s - loss: 0.0638 - categorical_accuracy: 0.9826Source path =  datasets/Project_data/val ; batch size = 40
Batch:  3 Index: 40

Epoch 00001: saving model to model_init_2021-07-0511_47_52.386877/model-00001-0.06377-0.98261-0.43489-0.86250.h5
67/67 [==============================] - 194s 3s/step - loss: 0.0638 - categorical_accuracy: 0.9826 - val_loss: 0.4349 - val_categorical_accuracy: 0.8625
Epoch 2/20
67/67 [==============================] - ETA: 0s - loss: 0.0620 - categorical_accuracy: 0.9811
Epoch 00002: saving model to model_init_2021-07-0511_47_52.386877/model-00002-0.06203-0.98112-0.41550-0.86500.h5
67/67 [==============================] - 134s 2s/step - loss: 0.0620 - categorical_accuracy: 0.9811 - val_loss: 0.4155 - val_categorical_accuracy: 0.8650
Epoch 3/20
67/67 [==============================] - ETA: 0s - loss: 0.0688 - categorical_accuracy: 0.9807
Epoch 00

Model C Experiment

In [34]:
model_c = Sequential()
model_c.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x,y,z,channel), padding="same"))
model_c.add(Activation('relu'))
model_c.add(Conv3D(32, padding="same", kernel_size=(3, 3, 3)))
model_c.add(Activation('relu'))
model_c.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_c.add(Dropout(0.25))

model_c.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_c.add(Activation('relu'))
model_c.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_c.add(Activation('relu'))
model_c.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_c.add(Dropout(0.25))

model_c.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_c.add(Activation('relu'))
model_c.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_c.add(Activation('relu'))
model_c.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_c.add(Dropout(0.25))

model_c.add(Flatten())
model_c.add(Dense(512, activation='relu'))
model_c.add(BatchNormalization())
model_c.add(Dropout(0.5))
model_c.add(Dense(classes, activation='softmax'))
model_c.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_c.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_12 (Conv3D)           (None, 30, 60, 60, 32)    2624      
_________________________________________________________________
activation_12 (Activation)   (None, 30, 60, 60, 32)    0         
_________________________________________________________________
conv3d_13 (Conv3D)           (None, 30, 60, 60, 32)    27680     
_________________________________________________________________
activation_13 (Activation)   (None, 30, 60, 60, 32)    0         
_________________________________________________________________
max_pooling3d_8 (MaxPooling3 (None, 10, 20, 20, 32)    0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 10, 20, 20, 32)    0         
_________________________________________________________________
conv3d_14 (Conv3D)           (None, 10, 20, 20, 64)   

In [35]:
# Changing back batch size, images per frame, height and width of image
batch_size = 10
x = 30 # number of frames
y = 120 # image width
z = 120# image height
num_epochs = 20

In [36]:
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)

In [38]:
model_c.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/20


InvalidArgumentError:  Input to reshape is a tensor with 32000 values, but the requested shape requires a multiple of 1152
	 [[node sequential_3/flatten_3/Reshape (defined at <ipython-input-37-31a23c77d6f4>:3) ]] [Op:__inference_train_function_31852]

Function call stack:
train_function


Model D Experiment

In [16]:
input_shape=(x,y,z,channel)

nb_filters = [8,16,32,64]
nb_dense = [1000, 500, 5]
# Define model
model_d = Sequential()

model_d.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_d.add(BatchNormalization())
model_d.add(Activation('relu'))

model_d.add(MaxPooling3D(pool_size=(2,2,2)))

model_d.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_d.add(BatchNormalization())
model_d.add(Activation('relu'))

model_d.add(MaxPooling3D(pool_size=(2,2,2)))

model_d.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_d.add(BatchNormalization())
model_d.add(Activation('relu'))

model_d.add(MaxPooling3D(pool_size=(2,2,2)))

model_d.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_d.add(BatchNormalization())
model_d.add(Activation('relu'))

model_d.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_d.add(Flatten())

model_d.add(Dense(nb_dense[0], activation='relu'))
model_d.add(Dropout(0.5))

model_d.add(Dense(nb_dense[1], activation='relu'))
model_d.add(Dropout(0.5))

#softmax layer
model_d.add(Dense(nb_dense[2], activation='softmax'))
model_d.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_d.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_4 (Conv3D)            (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_4 (Batch (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_4 (Activation)    (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_4 (MaxPooling3 (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_5 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_5 (Batch (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_5 (Activation)    (None, 15, 60, 60, 16)   

In [40]:
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)
num_epochs = 10
model_d.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  datasets/Project_data/train ; batch size = 10
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 2.9367 - categorical_accuracy: 0.2217Source path =  datasets/Project_data/val ; batch size = 10

Epoch 00001: saving model to model_init_2021-07-0511_47_52.386877/model-00001-2.93669-0.22172-1.60484-0.24000.h5
67/67 [==============================] - 91s 1s/step - loss: 2.9367 - categorical_accuracy: 0.2217 - val_loss: 1.6048 - val_categorical_accuracy: 0.2400
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 2.5916 - categorical_accuracy: 0.2637
Epoch 00002: saving model to model_init_2021-07-0511_47_52.386877/model-00002-2.59165-0.26368-1.61963-0.17000.h5
67/67 [==============================] - 39s 582ms/step - loss: 2.5916 - categorical_accuracy: 0.2637 - val_loss: 1.6196 - val_categorical_accuracy: 0.1700
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 2.6942 - categorical_accuracy: 0.2189
Epoch 00003: saving model t

Changing Model with images per frame, width and height of the images Experiment

In [13]:

x = 30 # number of frames
y = 64 # image width
z = 64 # image height 

classes = 5
channel = 1

In [14]:
model_e = Sequential()
model_e.add(Conv3D(32, kernel_size=(3, 3, 3), input_shape=(x,y,z,channel), padding="same"))
model_e.add(Activation('relu'))
model_e.add(Conv3D(32, padding="same", kernel_size=(3, 3, 3)))
model_e.add(Activation('relu'))
model_e.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_e.add(Dropout(0.25))

model_e.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_e.add(Activation('relu'))
model_e.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_e.add(Activation('relu'))
model_e.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_e.add(Dropout(0.25))

model_e.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_e.add(Activation('relu'))
model_e.add(Conv3D(64, padding="same", kernel_size=(3, 3, 3)))
model_e.add(Activation('relu'))
model_e.add(MaxPooling3D(pool_size=(3, 3, 3), padding="same"))
model_e.add(Dropout(0.25))

model_e.add(Flatten())
model_e.add(Dense(512, activation='relu'))
model_e.add(BatchNormalization())
model_e.add(Dropout(0.5))
model_e.add(Dense(classes, activation='softmax'))
model_e.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_e.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_8 (Conv3D)            (None, 30, 64, 64, 32)    896       
_________________________________________________________________
activation_8 (Activation)    (None, 30, 64, 64, 32)    0         
_________________________________________________________________
conv3d_9 (Conv3D)            (None, 30, 64, 64, 32)    27680     
_________________________________________________________________
activation_9 (Activation)    (None, 30, 64, 64, 32)    0         
_________________________________________________________________
max_pooling3d_8 (MaxPooling3 (None, 10, 22, 22, 32)    0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 10, 22, 22, 32)    0         
_________________________________________________________________
conv3d_10 (Conv3D)           (None, 10, 22, 22, 64)   

In [43]:
batch_size = 5
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)
num_epochs = 20
model_e.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  datasets/Project_data/train ; batch size = 5
Epoch 1/20
67/67 [==============================] - ETA: 0s - loss: 1.9819 - categorical_accuracy: 0.1672Source path =  datasets/Project_data/val ; batch size = 5

Epoch 00001: saving model to model_init_2021-07-0511_47_52.386877/model-00001-1.98186-0.16716-1.61269-0.16000.h5
67/67 [==============================] - 39s 580ms/step - loss: 1.9819 - categorical_accuracy: 0.1672 - val_loss: 1.6127 - val_categorical_accuracy: 0.1600
Epoch 2/20
67/67 [==============================] - ETA: 0s - loss: 1.9845 - categorical_accuracy: 0.1903
Epoch 00002: saving model to model_init_2021-07-0511_47_52.386877/model-00002-1.98450-0.19033-1.61574-0.20000.h5
67/67 [==============================] - 39s 583ms/step - loss: 1.9845 - categorical_accuracy: 0.1903 - val_loss: 1.6157 - val_categorical_accuracy: 0.2000
Epoch 3/20
67/67 [==============================] - ETA: 0s - loss: 1.9650 - categorical_accuracy: 0.1791
Epoch 00003: saving model 

Model F Experiment

In [15]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height 

classes = 5
channel = 1
nb_dense = [1000, 500, 5]

input_shape=(x,y,z,channel)

# Define model
model_f = Sequential()

model_f.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_f.add(BatchNormalization())
model_f.add(Activation('relu'))

model_f.add(MaxPooling3D(pool_size=(2,2,2)))

model_f.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_f.add(BatchNormalization())
model_f.add(Activation('relu'))

model_f.add(MaxPooling3D(pool_size=(2,2,2)))

model_f.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_f.add(BatchNormalization())
model_f.add(Activation('relu'))

model_f.add(MaxPooling3D(pool_size=(2,2,2)))

model_f.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_f.add(BatchNormalization())
model_f.add(Activation('relu'))

model_f.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_f.add(Flatten())

model_f.add(Dense(nb_dense[0], activation='relu'))
model_f.add(Dropout(0.5))

model_f.add(Dense(nb_dense[1], activation='relu'))
model_f.add(Dropout(0.5))

#softmax layer
model_f.add(Dense(nb_dense[2], activation='softmax'))
model_f.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_f.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_14 (Conv3D)           (None, 30, 120, 120, 8)   224       
_________________________________________________________________
batch_normalization_9 (Batch (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_14 (Activation)   (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_11 (MaxPooling (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_15 (Conv3D)           (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_10 (Batc (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_15 (Activation)   (None, 15, 60, 60, 16)   

In [45]:
batch_size = 10
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)
num_epochs = 20
model_f.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  datasets/Project_data/train ; batch size = 10
Epoch 1/20
67/67 [==============================] - ETA: 0s - loss: 2.9727 - categorical_accuracy: 0.2262Source path =  datasets/Project_data/val ; batch size = 10

Epoch 00001: saving model to model_init_2021-07-0511_47_52.386877/model-00001-2.97274-0.22624-1.62609-0.17000.h5
67/67 [==============================] - 83s 1s/step - loss: 2.9727 - categorical_accuracy: 0.2262 - val_loss: 1.6261 - val_categorical_accuracy: 0.1700
Epoch 2/20
67/67 [==============================] - ETA: 0s - loss: 2.8852 - categorical_accuracy: 0.2338
Epoch 00002: saving model to model_init_2021-07-0511_47_52.386877/model-00002-2.88525-0.23383-1.64439-0.20000.h5
67/67 [==============================] - 38s 568ms/step - loss: 2.8852 - categorical_accuracy: 0.2338 - val_loss: 1.6444 - val_categorical_accuracy: 0.2000
Epoch 3/20
67/67 [==============================] - ETA: 0s - loss: 2.8538 - categorical_accuracy: 0.2488
Epoch 00003: saving model t

Model G Experiment

In [16]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height 

classes = 5
channel = 3
nb_dense = [1000, 500, 5]

input_shape=(x,y,z,channel)

# Define model
model_g = Sequential()

model_g.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_g.add(BatchNormalization())
model_g.add(Activation('relu'))

model_g.add(MaxPooling3D(pool_size=(2,2,2)))

model_g.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_g.add(BatchNormalization())
model_g.add(Activation('relu'))

model_g.add(MaxPooling3D(pool_size=(2,2,2)))

model_g.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_g.add(BatchNormalization())
model_g.add(Activation('relu'))

model_g.add(MaxPooling3D(pool_size=(2,2,2)))

model_g.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_g.add(BatchNormalization())
model_g.add(Activation('relu'))

model_g.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_g.add(Flatten())

model_g.add(Dense(nb_dense[0], activation='relu'))
model_g.add(Dropout(0.5))

model_g.add(Dense(nb_dense[1], activation='relu'))
model_g.add(Dropout(0.5))

#softmax layer
model_g.add(Dense(nb_dense[2], activation='softmax'))
model_g.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_g.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_18 (Conv3D)           (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_13 (Batc (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_18 (Activation)   (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_15 (MaxPooling (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_19 (Conv3D)           (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_14 (Batc (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_19 (Activation)   (None, 15, 60, 60, 16)   

In [47]:
batch_size = 10
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)
num_epochs = 10
model_g.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  datasets/Project_data/train ; batch size = 10
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 3.4285 - categorical_accuracy: 0.1900Source path =  datasets/Project_data/val ; batch size = 10

Epoch 00001: saving model to model_init_2021-07-0511_47_52.386877/model-00001-3.42847-0.19005-1.65610-0.26000.h5
67/67 [==============================] - 92s 1s/step - loss: 3.4285 - categorical_accuracy: 0.1900 - val_loss: 1.6561 - val_categorical_accuracy: 0.2600
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 3.1976 - categorical_accuracy: 0.1940
Epoch 00002: saving model to model_init_2021-07-0511_47_52.386877/model-00002-3.19758-0.19403-1.74571-0.26000.h5
67/67 [==============================] - 37s 553ms/step - loss: 3.1976 - categorical_accuracy: 0.1940 - val_loss: 1.7457 - val_categorical_accuracy: 0.2600
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 3.0655 - categorical_accuracy: 0.2289
Epoch 00003: saving model t

Model H Experiment

In [17]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height 

classes = 5
channel = 3

input_shape=(x,y,z,channel)

# Define model
model_h = Sequential()

model_h.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_h.add(BatchNormalization())
model_h.add(Activation('relu'))

model_h.add(MaxPooling3D(pool_size=(2,2,2)))

model_h.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_h.add(BatchNormalization())
model_h.add(Activation('relu'))

model_h.add(MaxPooling3D(pool_size=(2,2,2)))

model_h.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_h.add(BatchNormalization())
model_h.add(Activation('relu'))

model_h.add(MaxPooling3D(pool_size=(2,2,2)))

model_h.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_h.add(BatchNormalization())
model_h.add(Activation('relu'))

model_h.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_h.add(Flatten())

model_h.add(Dense(nb_dense[0], activation='relu'))
model_h.add(Dropout(0.5))

model_h.add(Dense(nb_dense[1], activation='relu'))
model_h.add(Dropout(0.5))

#softmax layer
model_h.add(Dense(nb_dense[2], activation='softmax'))
model_h.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_h.summary()


Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_22 (Conv3D)           (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_17 (Batc (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_22 (Activation)   (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_19 (MaxPooling (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_23 (Conv3D)           (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_18 (Batc (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_23 (Activation)   (None, 15, 60, 60, 16)   

In [38]:
batch_size = 10
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)
num_epochs = 10
model_h.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Instructions for updating:
Please use Model.fit, which supports generators.
Source path =  datasets/Project_data/train ; batch size = 10
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 3.0449 - categorical_accuracy: 0.2202Source path =  datasets/Project_data/val ; batch size = 10

Epoch 00001: saving model to model_init_2021-07-0514_52_19.218304/model-00001-3.04492-0.22021-1.58914-0.21000.h5
67/67 [==============================] - 91s 1s/step - loss: 3.0449 - categorical_accuracy: 0.2202 - val_loss: 1.5891 - val_categorical_accuracy: 0.2100
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 2.8164 - categorical_accuracy: 0.2637
Epoch 00002: saving model to model_init_2021-07-0514_52_19.218304/model-00002-2.81640-0.26368-1.56729-0.22000.h5
67/67 [==============================] - 38s 574ms/step - loss: 2.8164 - categorical_accuracy: 0.2637 - val_loss: 1.5673 - val_categorical_accuracy: 0.2200
Epoch 3/10
67/67 [==============================] - ETA: 

Model I Experiment

In [17]:
x = 30 # number of frames
y = 120 # image width
z = 120 # image height 

classes = 5
channel = 3

input_shape=(x,y,z,channel)

# Define model
model_i = Sequential()

model_i.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_i.add(BatchNormalization())
model_i.add(Activation('relu'))

model_i.add(MaxPooling3D(pool_size=(2,2,2)))

model_i.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_i.add(BatchNormalization())
model_i.add(Activation('relu'))

model_i.add(MaxPooling3D(pool_size=(2,2,2)))

model_i.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_i.add(BatchNormalization())
model_i.add(Activation('relu'))

model_i.add(MaxPooling3D(pool_size=(2,2,2)))

model_i.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_i.add(Activation('relu'))
model_i.add(Dropout(0.25))

model_i.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_i.add(Flatten())

model_i.add(Dense(nb_dense[0], activation='relu'))
model_i.add(Dropout(0.5))

model_i.add(Dense(nb_dense[1], activation='relu'))
model_i.add(Dropout(0.5))

#softmax layer
model_i.add(Dense(nb_dense[2], activation='softmax'))
model_i.compile(optimizer=keras.optimizers.Adadelta(), loss='categorical_crossentropy', metrics=['categorical_accuracy'])
model_i.summary()


Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_8 (Conv3D)            (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_8 (Batch (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_8 (Activation)    (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_8 (MaxPooling3 (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_9 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_9 (Batch (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_9 (Activation)    (None, 15, 60, 60, 16)   

In [20]:
batch_size = 10
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)
num_epochs = 10
model_i.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  datasets/Project_data/train ; batch size = 10
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 4.2111 - categorical_accuracy: 0.1855Source path =  datasets/Project_data/val ; batch size = 10

Epoch 00001: saving model to model_init_2021-07-0515_14_59.816322/model-00001-4.21114-0.18552-1.66875-0.20000.h5
67/67 [==============================] - 90s 1s/step - loss: 4.2111 - categorical_accuracy: 0.1855 - val_loss: 1.6687 - val_categorical_accuracy: 0.2000
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 3.8889 - categorical_accuracy: 0.1841
Epoch 00002: saving model to model_init_2021-07-0515_14_59.816322/model-00002-3.88888-0.18408-1.69613-0.20000.h5
67/67 [==============================] - 38s 571ms/step - loss: 3.8889 - categorical_accuracy: 0.1841 - val_loss: 1.6961 - val_categorical_accuracy: 0.2000
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 3.8231 - categorical_accuracy: 0.2587
Epoch 00003: saving model t

Final Model

In [20]:
nb_filters = [8,16,32,64]
nb_dense = [256, 128, 5]

# Input
input_shape=(30,120,120,3)

# Define model
model_final = Sequential()

model_final.add(Conv3D(nb_filters[0], 
                 kernel_size=(3,3,3), 
                 input_shape=input_shape,
                 padding='same'))
model_final.add(BatchNormalization())
model_final.add(Activation('relu'))

model_final.add(MaxPooling3D(pool_size=(2,2,2)))

model_final.add(Conv3D(nb_filters[1], 
                 kernel_size=(3,3,3), 
                 padding='same'))
model_final.add(BatchNormalization())
model_final.add(Activation('relu'))

model_final.add(MaxPooling3D(pool_size=(2,2,2)))

model_final.add(Conv3D(nb_filters[2], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_final.add(BatchNormalization())
model_final.add(Activation('relu'))

model_final.add(MaxPooling3D(pool_size=(2,2,2)))

model_final.add(Conv3D(nb_filters[3], 
                 kernel_size=(1,3,3), 
                 padding='same'))
model_final.add(Activation('relu'))
model_final.add(Dropout(0.25))

model_final.add(MaxPooling3D(pool_size=(2,2,2)))

#Flatten Layers
model_final.add(Flatten())

model_final.add(Dense(nb_dense[0], activation='relu'))
model_final.add(Dropout(0.5))

model_final.add(Dense(nb_dense[1], activation='relu'))
model_final.add(Dropout(0.5))

#softmax layer
model_final.add(Dense(nb_dense[2], activation='softmax'))

In [21]:
optimiser = keras.optimizers.Adam() #write your optimizer
model_final.compile(optimizer=optimiser, loss='categorical_crossentropy', metrics=['categorical_accuracy'])
print (model_final.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_8 (Conv3D)            (None, 30, 120, 120, 8)   656       
_________________________________________________________________
batch_normalization_7 (Batch (None, 30, 120, 120, 8)   32        
_________________________________________________________________
activation_8 (Activation)    (None, 30, 120, 120, 8)   0         
_________________________________________________________________
max_pooling3d_8 (MaxPooling3 (None, 15, 60, 60, 8)     0         
_________________________________________________________________
conv3d_9 (Conv3D)            (None, 15, 60, 60, 16)    3472      
_________________________________________________________________
batch_normalization_8 (Batch (None, 15, 60, 60, 16)    64        
_________________________________________________________________
activation_9 (Activation)    (None, 15, 60, 60, 16)   

In [19]:
train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)

In [24]:
batch_size = 10
num_epochs = 10
model_final.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  datasets/Project_data/train ; batch size = 10
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 1.9121 - categorical_accuracy: 0.2428Source path =  datasets/Project_data/val ; batch size = 10

Epoch 00001: saving model to model_init_2021-07-0515_14_59.816322/model-00001-1.91214-0.24284-1.60231-0.23000.h5
67/67 [==============================] - 93s 1s/step - loss: 1.9121 - categorical_accuracy: 0.2428 - val_loss: 1.6023 - val_categorical_accuracy: 0.2300
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 1.5850 - categorical_accuracy: 0.2886
Epoch 00002: saving model to model_init_2021-07-0515_14_59.816322/model-00002-1.58498-0.28856-1.60293-0.25000.h5
67/67 [==============================] - 42s 621ms/step - loss: 1.5850 - categorical_accuracy: 0.2886 - val_loss: 1.6029 - val_categorical_accuracy: 0.2500
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 1.5996 - categorical_accuracy: 0.2289
Epoch 00003: saving model t

In [ ]:
batch_size = 32
num_epochs = 10
model_final.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 1.4646 - categorical_accuracy: 0.2935
Epoch 00001: saving model to model_init_2021-07-0515_14_59.816322/model-00001-1.46459-0.29353-1.36525-0.43000.h5
67/67 [==============================] - 40s 603ms/step - loss: 1.4646 - categorical_accuracy: 0.2935 - val_loss: 1.3652 - val_categorical_accuracy: 0.4300
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 1.3757 - categorical_accuracy: 0.3930
Epoch 00002: saving model to model_init_2021-07-0515_14_59.816322/model-00002-1.37567-0.39303-1.37942-0.47000.h5
67/67 [==============================] - 40s 600ms/step - loss: 1.3757 - categorical_accuracy: 0.3930 - val_loss: 1.3794 - val_categorical_accuracy: 0.4700
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 1.3293 - categorical_accuracy: 0.4378
Epoch 00003: saving model to model_init_2021-07-0515_14_59.816322/model-00003-1.32927-0.43781-1.43511-0.39000.h5

Epoch 00003: ReduceLROnPlateau

In [22]:
batch_size = 64
num_epochs = 10

train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)


model_final.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  datasets/Project_data/train ; batch size = 64
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 1.8006 - categorical_accuracy: 0.3034Source path =  datasets/Project_data/val ; batch size = 64
Batch:  2 Index: 64
Batch:  3 Index: 36
Batch:  4 Index: 28

Epoch 00001: saving model to model_init_2021-07-0516_22_40.984452/model-00001-1.80063-0.30343-1.51209-0.30696.h5
67/67 [==============================] - 245s 4s/step - loss: 1.8006 - categorical_accuracy: 0.3034 - val_loss: 1.5121 - val_categorical_accuracy: 0.3070
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 1.3279 - categorical_accuracy: 0.4302Batch:  7 Index: 16

Epoch 00002: saving model to model_init_2021-07-0516_22_40.984452/model-00002-1.32793-0.43018-1.54591-0.23864.h5
67/67 [==============================] - 144s 2s/step - loss: 1.3279 - categorical_accuracy: 0.4302 - val_loss: 1.5459 - val_categorical_accuracy: 0.2386
Epoch 3/10
67/67 [==============================] - ET

In [23]:
batch_size = 40
num_epochs = 10

train_generator = generator_ex(train_path, train_doc, batch_size)
val_generator = generator_ex(val_path, val_doc, batch_size)


model_final.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list, validation_data=val_generator, 
                    validation_steps=validation_steps, class_weight=None, workers=1, initial_epoch=0)

Source path =  datasets/Project_data/train ; batch size = 40
Epoch 1/10
67/67 [==============================] - ETA: 0s - loss: 0.2173 - categorical_accuracy: 0.9217Source path =  datasets/Project_data/val ; batch size = 40
Batch:  3 Index: 40

Epoch 00001: saving model to model_init_2021-07-0516_22_40.984452/model-00001-0.21730-0.92174-0.92560-0.67917.h5
67/67 [==============================] - 229s 3s/step - loss: 0.2173 - categorical_accuracy: 0.9217 - val_loss: 0.9256 - val_categorical_accuracy: 0.6792
Epoch 2/10
67/67 [==============================] - ETA: 0s - loss: 0.1964 - categorical_accuracy: 0.9356
Epoch 00002: saving model to model_init_2021-07-0516_22_40.984452/model-00002-0.19638-0.93562-0.53911-0.81500.h5
67/67 [==============================] - 161s 2s/step - loss: 0.1964 - categorical_accuracy: 0.9356 - val_loss: 0.5391 - val_categorical_accuracy: 0.8150
Epoch 3/10
67/67 [==============================] - ETA: 0s - loss: 0.1748 - categorical_accuracy: 0.9412
Epoch 00